In [9]:
import numpy as np
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import r2_score

from pathlib import Path

import yaml
from yaml import Loader

In [2]:
# path to dataset
# change to reflect dataset location

train_path = 'split_set/train'
test_path = 'split_set/test'
val_path = 'split_set/val'

architecture_vocab = ['Conv2d', 'Linear', 'MaxPool2d', 'BatchNorm2d', 'Dropout2d', 'ReLU', 'SELU', 'LeakyReLU', 'Flatten', 'Tanh', 'BatchNorm1d', 'Dropout', 'Softmax']
# unnecessary as all models use Adam
optimizer_vocab = ['SGD', 'Adam', 'Adadelta', 'Adagrad']

In [3]:
# load training set

train_data = []

for filename in Path(train_path).glob('**/meta_data.yml'):
    data_item = {}
    with open(filename, 'r') as f:
        yamldata = yaml.load(f, Loader=Loader)
        data_item['mdl_str'] = yamldata['arch_and_hp']
        data_item['train_loss'] = yamldata['train_loss']
        data_item['train_error'] = yamldata['train_error']
        data_item['val_loss'] = yamldata['val_loss']
        data_item['val_error'] = yamldata['val_error']
        data_item['test_loss'] = yamldata['test_loss']
        data_item['test_error'] = yamldata['test_error']
        # model architecture as bag of words
        data_item['mdl_vect'] = [vocab in data_item['mdl_str'] for vocab in architecture_vocab]
    
    train_data.append(data_item)
    
train_data = np.array(train_data)

train_X = []
train_y = []

for data_item in train_data:
    train_X.append(data_item['mdl_vect'])
    train_y.append(data_item['test_error'])
    
train_X = np.array(train_X, dtype=int)
train_y = np.array(train_y, dtype=float)

In [4]:
# load test set

test_data = []

for filename in Path(test_path).glob('**/meta_data.yml'):
    data_item = {}
    with open(filename, 'r') as f:
        yamldata = yaml.load(f, Loader=Loader)
        data_item['mdl_str'] = yamldata['arch_and_hp']
        data_item['train_loss'] = yamldata['train_loss']
        data_item['train_error'] = yamldata['train_error']
        data_item['val_loss'] = yamldata['val_loss']
        data_item['val_error'] = yamldata['val_error']
        data_item['test_loss'] = yamldata['test_loss']
        data_item['test_error'] = yamldata['test_error']
        # model architecture as bag of words
        data_item['mdl_vect'] = [vocab in data_item['mdl_str'] for vocab in architecture_vocab]
    
    test_data.append(data_item)
    
test_data = np.array(test_data)

test_X = []
test_y = []

for data_item in test_data:
    test_X.append(data_item['mdl_vect'])
    test_y.append(data_item['test_error'])
    
test_X = np.array(test_X, dtype=int)
test_y = np.array(test_y, dtype=float)

In [5]:
# load validation set

val_data = []

for filename in Path(val_path).glob('**/meta_data.yml'):
    data_item = {}
    with open(filename, 'r') as f:
        yamldata = yaml.load(f, Loader=Loader)
        data_item['mdl_str'] = yamldata['arch_and_hp']
        data_item['train_loss'] = yamldata['train_loss']
        data_item['train_error'] = yamldata['train_error']
        data_item['val_loss'] = yamldata['val_loss']
        data_item['val_error'] = yamldata['val_error']
        data_item['test_loss'] = yamldata['test_loss']
        data_item['test_error'] = yamldata['test_error']
        # model architecture as bag of words
        data_item['mdl_vect'] = [vocab in data_item['mdl_str'] for vocab in architecture_vocab]
    
    val_data.append(data_item)
    
val_data = np.array(val_data)

val_X = []
val_y = []

for data_item in val_data:
    val_X.append(data_item['mdl_vect'])
    val_y.append(data_item['test_error'])
    
val_X = np.array(val_X, dtype=int)
val_y = np.array(val_y, dtype=float)

In [6]:
reg = LinearRegression().fit(train_X, train_y)
ridge = Ridge(alpha=1.0).fit(train_X, train_y)
knn = KNeighborsRegressor(n_neighbors=25).fit(train_X, train_y)

In [12]:
test_y_reg = reg.predict(test_X)
reg_sse_err = np.sum((test_y_reg - test_y) ** 2)
reg_mse_err = reg_sse_err / test_X.shape[0]
reg_r2 = r2_score(test_y, test_y_reg)

test_y_ridge = ridge.predict(test_X)
ridge_sse_err = np.sum((test_y_ridge - test_y) ** 2)
ridge_mse_err = ridge_sse_err / test_X.shape[0]
ridge_r2 = r2_score(test_y, test_y_ridge)

test_y_knn = knn.predict(test_X)
knn_sse_err = np.sum((test_y_knn - test_y) ** 2)
knn_mse_err = knn_sse_err / test_X.shape[0]
knn_r2 = r2_score(test_y, test_y_knn)

print("Sum of Squared Errors for:")
print("Linear Regression, Ridge Regression, KNN")
print(reg_sse_err, ridge_sse_err, knn_sse_err)
print()
print("Mean Squared Errors for:")
print("Linear Regression, Ridge Regression, KNN")
print(reg_mse_err, ridge_mse_err, knn_mse_err)
print()
print("R2 Scores for :")
print("Linear Regression, Ridge Regression, KNN")
print(reg_r2, ridge_r2, knn_r2)

Sum of Squared Errors for:
Linear Regression, Ridge Regression, KNN
2.280754121139546 2.2809928846553778 2.1681026066284175

Mean Squared Errors for:
Linear Regression, Ridge Regression, KNN
0.009463710046222183 0.009464700766204887 0.008996276376051524

R2 Scores for :
Linear Regression, Ridge Regression, KNN
0.19982641540232382 0.19974264825856924 0.23934876695317364
